# 한글 문서를 위한 스키마 정의 후 Vector Store 생성 

이 노트북은 오픈 서치 인덱스를 정의하고, 한글 전체 문서를 청킹하여 인덱스를 생성을 하는 노트북 입니다. 이후에 렉시컬, 시멘틱, 하이브리드 검색을 통하여 인덱스의 동작을 확인 합니다. 
이후에 전체 문서의 일부를 검증 인덱스로 생성하여, 검증용으로 사용을 합니다.


---

## [중요] 사전 실행 노트북
이 노트북은 아래 두개의 셋업 노트북이 먼저 실행이 되어야 합니다.
- (1) Setup 노트북
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup.ipynb 와 같습니다.
    -  [Setup Notebook](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup.ipynb)
- (2) Amazon OpenSearch 설치 노트북    
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb 와 같습니다.
    - [Setup OpenSearch](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb)

# 0. Setting
 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)


python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot']


# 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from local_utils import bedrock, print_ww
from local_utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


# 2. Titan Embedding 모델 로딩

In [4]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id=bedrock_info.get_model_id(
        model_name="Titan-Embeddings-G1"
    )
)
print("Bedrock Embeddings Model Loaded")

Bedrock Embeddings Model Loaded


# 3. Load all Json files

In [5]:
import glob
from local_utils.proc_docs import get_load_json, show_doc_json

In [6]:
# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/preprocessed_json/*data_ko.json'

# List all .txt files in the specified folder
json_files = glob.glob(folder_path)
# json_files = ['data/poc/customer_EFOTA.json']

# Load each item per json file and append to a list
doc_json_list = []
for file_path in json_files:
    print (json_files)
    doc_json = get_load_json(file_path)
    doc_json_list.append(doc_json)

print("all json files: ", len(doc_json_list))    
# Flatten the list of lists into a single list
all_docs = []
for item in doc_json_list:
    all_docs.extend(item)
        
print("all items: ", len(all_docs))

['data/poc/preprocessed_json/all_processed_data_ko.json']
.[]
all json files:  1
all items:  1732


In [7]:
all_docs[0:2]

[Document(page_content='사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': '사용 방법 동영상', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27'}),
 Document(page_content='녹스 E-FOTA.Knox E-FOTA를 사용하면 기업 IT 관리자가 사용자 개입 없이 OS 버전과 보안 업데이트를 기업 디바이스에 원격으로 배포할 수 있습니다.배포 전에 업데이트를 테스트하여 사내 앱과 새 OS 버전 간의 호환성을 확인하고, 최신 보안 패치를 일정에 따라 배포하여 엔터프라이즈 디바이스의 보안을 강화하세요.대상 고객 무료 체험하기 이 문서의 대상: 시스템 보안 설계자 — Knox E-FOTA의 작동 원리와 Knox E-FOTA를 사용하여 엔터프라이즈 디바이스를 업데이트하는 방법을 알아보세요.IT 관리자 - 엔터프라이즈 디바이스의 무선 업데이트를 관리하는 방법

# 4. Chunking JSON Doc 

## Chunk Size and Chunk Overlap Size 결정

In [8]:
chunk_size = 1024
chunk_overlap = 512

## Chunking

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SpacyTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

chunk_docs = text_splitter.split_documents(all_docs)
print(f"Number of chunk_docs after split and chunking= {len(chunk_docs)}")

Number of chunk_docs after split and chunking= 4909


In [10]:
chunk_docs[0:2]

[Document(page_content='사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': '사용 방법 동영상', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27'}),
 Document(page_content='녹스 E-FOTA.Knox E-FOTA를 사용하면 기업 IT 관리자가 사용자 개입 없이 OS 버전과 보안 업데이트를 기업 디바이스에 원격으로 배포할 수 있습니다.배포 전에 업데이트를 테스트하여 사내 앱과 새 OS 버전 간의 호환성을 확인하고, 최신 보안 패치를 일정에 따라 배포하여 엔터프라이즈 디바이스의 보안을 강화하세요.대상 고객 무료 체험하기 이 문서의 대상: 시스템 보안 설계자 — Knox E-FOTA의 작동 원리와 Knox E-FOTA를 사용하여 엔터프라이즈 디바이스를 업데이트하는 방법을 알아보세요.IT 관리자 - 엔터프라이즈 디바이스의 무선 업데이트를 관리하는 방법

## Sampling

In [11]:
len(chunk_docs)
chunk_docs = chunk_docs[:1000]
len(chunk_docs)

1000

# 5. Index 생성

## Index 이름 결정
- 아래에 원하는 인덱스 이름을 기술 해주세요

In [12]:
# index_name = "<your index name>"
index_name = "knox-korean-v22"

## Index 스키마 정의

In [13]:
index_body = {
    'settings': {
        'analysis': {'analyzer': {'my_analyzer': {'char_filter': ['html_strip'],
                                                    'tokenizer': 'nori',
                                                       'filter': [
                                                                   'my_nori_part_of_speech'
                                                                 ],
                                                       'type': 'custom'}},
                                   'tokenizer': {'nori': {
                                                  'decompound_mode': 'mixed',
                                                  'discard_punctuation': 'true',
                                                  'type': 'nori_tokenizer'}
                                                },
                                    "filter": {
                                          "my_nori_part_of_speech": {
                                                "type": "nori_part_of_speech",
                                                "stoptags": [
                                                    "J", "XSV", "E", "IC","MAJ","NNB",
                                                    "SP", "SSC", "SSO",
                                                    "SC","SE","XSN","XSV",
                                                    "UNA","NA","VCP","VSV",
                                                    "VX"
                                                ]
                                          }
                                    }
                    },        
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                               'source' : {'type': 'keyword'},                    
                               'last_updated': {'type': 'date'},
                               'project': {'type': 'keyword'},
                               'seq_num': {'type': 'long'},
                               'title': {'type': 'text'},  # For full-text search
                               'url': {'type': 'text'},  # For full-text search
                            }
            },            
            'text': {
                'analyzer': 'my_analyzer',
                'search_analyzer': 'my_analyzer',
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': 1536  # Replace with your vector dimension
            }
        }
    }
}


# 5. LangChain OpenSearch VectorStore 생성 
## 선수 조건


## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [20]:
import boto3
from local_utils.proc_docs import get_parameter

In [21]:
ssm = boto3.client('ssm', 'us-east-1')

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)


In [22]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

## OpenSearch Client 생성

In [24]:
from local_utils.opensearch import opensearch_utils

In [25]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [26]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=knox-korean-v22, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'knox-korean-v22'}
Index is created
{'knox-korean-v22': {'aliases': {},
                     'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                             'project': {'type': 'keyword'},
                                                                             'seq_num': {'type': 'long'},
                                                                             'source': {'type': 'keyword'},
                                                                             'title': {'type': 'text'},
                                                                             'url': {'type': 'text'}}},
                                                 'text': {'analyzer': 'my_analyzer',
                                                          'type': 'text'},
       

## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [27]:
from langchain.vectorstores import OpenSearchVectorSearch

In [28]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60
)
vector_db

## OpenSearch 에 문서 삽입

In [29]:
%%time

vector_db.add_documents(
    documents = chunk_docs, 
    vector_field = "vector_field",
    bulk_size = 1000000
)


CPU times: user 2.28 s, sys: 129 ms, total: 2.41 s
Wall time: 1min 53s


['5e65f55b-76c7-40a3-aedb-75bd388ac130',
 'f5f0a5a0-7ce2-42a2-86e6-d066191a29c5',
 '131b3438-3393-4e30-9b06-eb8c644ce4d9',
 '5daea659-836e-4d9c-b7fc-ec0ecbceeede',
 '931c3081-0edb-427c-bb18-b90896ea5c3d',
 'cc4c3153-0da4-42d0-9655-540ae0ab088b',
 '835064b2-b258-4470-81d7-6d65fce9445c',
 'c01b0cbd-f5ee-4a47-8b1f-9d7dd5456fe4',
 'ba711a6b-7604-4fa8-a18a-2fc3694582ef',
 'dc3521fb-56db-4c52-8805-bb06e4a1b8bc',
 '8479ab32-bf09-4cd3-b991-f7fda44c26c6',
 '46f25098-65a8-48d0-bc2f-c796cc3cfb0a',
 '33da5ca7-1d16-44ac-8e98-8f4b4dcbe934',
 '391f399d-9195-40af-a5b8-26d4b0ef6542',
 'be2ab0af-edef-4e01-8a25-f272e75a59dd',
 'f5c5e1c3-74a3-4cc9-8484-dc229280945d',
 '9b6a5d81-b7df-466d-8e4d-60c697e7b782',
 '864137fe-a748-4518-8b4d-84acedd51e6c',
 '51525ccc-85c4-4918-aa7c-18e24f8ad5d6',
 '81bcc2ea-9196-44ac-8c38-f878aa7f7908',
 '16732413-cb9a-439f-8ee5-e78701af473c',
 'f9d0837c-b3c7-4175-873a-223c2674e4a9',
 '00c84d34-072e-4dd6-bec4-dc9b8333c6cc',
 '67f2fa62-7330-4b62-a419-2517f1be9731',
 '46bd7362-bd31-

# 6. 검색 테스트

In [34]:
from local_utils.rag import retriever_utils
from local_utils.rag import show_context_used

## Lexical 검색

In [35]:
query = "Knox란 무엇인가요?"

search_filter=[
    #{"term": {"metadata.seq_num": "256"}},
    #{"term": {"metadata.project": "KM"}},
]

In [36]:
similar_docs_lexical = retriever_utils.get_lexical_similar_docs(
    index_name=index_name,
    os_client=os_client,
    query=query,
    k=5,
    filter=search_filter
)
show_context_used(similar_docs_lexical)

-----------------------------------------------
1. Chunk: 163 Characters
-----------------------------------------------
슈퍼 어드민이란 무엇인가요?.슈퍼 관리자 또는 기본 관리자는 SamsungKnox.com, Knox 클라우드 서비스 또는 Knox 파트너 프로그램에 처음으로 등록한 사용자를
말합니다.최고 관리자는 추가 관리자를 초대하여 각 Knox 서비스의 관리 콘솔 내에서 기능을 보고 관리할 수 있습니다.
metadata:
 {'source': 'all_processed_data_ko.json', 'seq_num': 217, 'title': '슈퍼 어드민이란 무엇인가요?', 'url':
'https://docs.samsungknox.com/admin/fundamentals/knox-faq/#what-is-a-super-admin', 'project': 'USP',
'last_updated': '2023-07-26', 'id': 'd44583ef-a30c-45d6-a80e-79adf26e94c1'}
-----------------------------------------------
2. Chunk: 168 Characters
-----------------------------------------------
녹스 애셋 인텔리전스의 목적은 무엇입니까?Knox Asset Intelligence는 실행 가능한 가시성과 운영 통찰력을 제공하는 클라우드 기반 데이터 분석 솔루션입니다.세부
디바이스 사용에 대한 가시성을 통해 IT 관리자는 생산성을 극대화하고 안정성을 보장하며 자산 관리를 미리 계획할 수 있습니다.
metadata:
 {'source': 'all_processed_data_ko.json', 'seq_num': 310, 'title': '녹스 애셋 인텔리전스의 목적은 무엇인가요?', 'url':
'https://docs.samsungknox.com/admi

## 시멘틱 검색

In [37]:
query = "Knox란 무엇인가요?"

search_filter=[
    #{"term": {"metadata.seq_num": "256"}},
    #{"term": {"metadata.project": "KM"}},
]

In [38]:
similar_docs_semantic = retriever_utils.get_semantic_similar_docs(
    index_name=index_name,
    os_client=os_client,
    llm_emb=llm_emb,
    query=query,
    k=5,
    boolean_filter=search_filter,
    hybrid=False
)
show_context_used(similar_docs_semantic)

-----------------------------------------------
1. Chunk: 973 Characters
-----------------------------------------------
삼성 Knox 계정을 만드세요.가입하기 전에, 특히 디바이스를 해외에 등록할 계획인 IT 관리자인 경우 시작하기 전에 반드시 읽어보시기 바랍니다.Knox 계정 등록 삼성 Knox
계정은 무료이며 단 몇 분 만에 만들 수 있습니다.삼성 Knox 계정이 있으면 Knox 서비스에 등록하고 Knox 관리 포털에 액세스할 수 있습니다. Knox 관리 포털을 통해 각
Knox 서비스의 디바이스, 관리자, 라이선스를 쉽게 관리할 수 있습니다.삼성 Knox 계정을 등록하려면 계정을 삼성 계정에 연결해야 합니다.아직 삼성 계정이 없는 경우
SamsungKnox.com에 가입할 때 계정을 만들라는 메시지가 표시됩니다.Knox 계정을 만들려면: 1.삼성녹스닷컴으로 이동하세요. 2.오른쪽 상단에서 시작하기를 클릭하세요.
3.비즈니스 이메일에서 업무에 사용하는 이메일 주소를 입력합니다.미국 및 EU 서버 모두에서 장치를 관리하는 데 두 개의 계정이 필요한 경우 이메일 사용자 이름에 +us 또는
+eu 또는 인식할 수 있는 다른 태그를 추가하세요.이렇게 하면 별도의 이메일 주소를 만들지 않고도 두 지역 계정에 모두 쉽게 액세스할 수 있습니다.자세한 내용은 시작하기 전에를
참조하십시오. 4.액세스하려는 Knox 솔루션 (Knox Suite, Knox Configure, Knox Guard, 비즈니스용 삼성 케어+ 또는 기타 제품 및 서비스) 을
선택하세요.Knox 모바일 등록과 같은 개별 Knox Suite 서비스에 액세스하려면 Knox Suite를 선택하세요. 5.내 계정 생성을 클릭합니다.삼성 계정이 없는 경우 계정을
만들라는 메시지가 표시됩니다.삼성 계정을 생성한 후 수신한 확인 이메일을 열고 링크를 클릭합니다.그러면 자동으로 SamsungKnox.com으로 리디렉션되

## 하이브리드 검색

In [39]:
query = "Knox란 무엇인가요?"

search_filter=[
    #{"term": {"metadata.seq_num": "256"}},
    #{"term": {"metadata.project": "KM"}},
]

In [41]:
%%time
similar_docs_hybrid = retriever_utils.search_hybrid(
    query=query,
    k=5,
    index_name=index_name,
    os_client=os_client,
    filter=search_filter,
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[0.51, 0.49], # semantic, lexical
    async_mode=True,
    llm_emb=llm_emb,
    verbose=True
)

##############################
async_mode
##############################
True
##############################
reranker
##############################
False
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['삼성 Knox 계정을 만드세요.가입하기 전에, 특히 디바이스를 해외에 등록할 계획인 IT 관리자인 경우 시작하기 전에 반드시 읽어보시기 바랍니다.Knox 계정 등록 삼성 Knox 계정은 무료이며 단 몇 분 만에 만들 수 있습니다.삼성 Knox 계정이 있으면 Knox 서비스에 등록하고 Knox 관리 포털에 액세스할 수 있습니다. Knox 관리 포털을 통해 각 Knox 서비스의 디바이스, 관리자, 라이선스를 쉽게 관리할 수 있습니다.삼성 Knox 계정을 등록하려면 계정을 삼성 계정에 연결해야 합니다.아직 삼성 계정이 없는 경우 SamsungKnox.com에 가입할 때 계정을 만들라는 메시지가 표시됩니다.Knox 계정을 만들려면: 1.삼성녹스닷컴으로 이동하세요. 2.오른쪽 상단에서 시작하기를 클릭하세요. 3.비즈니스 이메일에서 업무에 사용하는 이메일 주소를 입력합니다.미국 및 EU 서버 모두에서 장치를 관리하는 데 두 개의 계정이 필요한 경우 이메일 사용자 이름에 +us 또는 +eu 또는 인식할 수 있는 다

# 7. 검증 인덱스 생성

## Index 이름 결정

In [46]:
eval_index_name = "genai-poc-knox-kr-eval-sample-0.02-1024c-256o-v23"

## Sampling

In [47]:
import random
def get_sampling_doc(seed, ratio, docs):

    random.seed(seed)
    
    eval_docs = docs[:int(len(docs)*ratio)]
    
    return eval_docs
    
eval_docs = get_sampling_doc(seed=200, ratio=0.02, docs= all_docs)
print("eval docs: ", len(eval_docs))
eval_docs[0:2]
    
    

eval docs:  34


[Document(page_content='사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.이 섹션에는 Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있습니다.Knox E-FOTA 시작하기 이 동영상은 Knox E-FOTA 콘솔을 안내하며 리셀러 등록, 디바이스 승인, 캠페인 생성, 캠페인 할당, 디바이스 상태 모니터링 방법을 보여줍니다.Knox E-FOTA에서 캠페인 만들기 다음 동영상은 삼성 디바이스에 Knox E-FOTA 캠페인을 만들고 적용하는 방법에 대한 심층적인 정보를 제공합니다.Knox E-FOTA를 VMware Workspace ONE에 연결하기 다음 비디오에서는 Knox E-FOTA를 VMware Workspace ONE과 연결하는 동시에 워크스페이스 1에서 디바이스 그룹을 추가하는 간단한 단계를 설명합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': '사용 방법 동영상', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27'}),
 Document(page_content='녹스 E-FOTA.Knox E-FOTA를 사용하면 기업 IT 관리자가 사용자 개입 없이 OS 버전과 보안 업데이트를 기업 디바이스에 원격으로 배포할 수 있습니다.배포 전에 업데이트를 테스트하여 사내 앱과 새 OS 버전 간의 호환성을 확인하고, 최신 보안 패치를 일정에 따라 배포하여 엔터프라이즈 디바이스의 보안을 강화하세요.대상 고객 무료 체험하기 이 문서의 대상: 시스템 보안 설계자 — Knox E-FOTA의 작동 원리와 Knox E-FOTA를 사용하여 엔터프라이즈 디바이스를 업데이트하는 방법을 알아보세요.IT 관리자 - 엔터프라이즈 디바이스의 무선 업데이트를 관리하는 방법

In [48]:
chunk_docs = text_splitter.split_documents(eval_docs)
print(f"Number of chunk_docs after split and chunking= {len(chunk_docs)}")

Number of chunk_docs after split and chunking= 90


## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [49]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    eval_index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        eval_index_name
    )
    
opensearch_utils.create_index(os_client, eval_index_name, index_body)
index_info = os_client.indices.get(index=eval_index_name)
print("Index is created")
pprint(index_info)    

index_name=genai-poc-knox-kr-eval-sample-0.02-1024c-256o-v23, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'genai-poc-knox-kr-eval-sample-0.02-1024c-256o-v23'}
Index is created
{'genai-poc-knox-kr-eval-sample-0.02-1024c-256o-v23': {'aliases': {},
                                                       'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                               'project': {'type': 'keyword'},
                                                                                                               'seq_num': {'type': 'long'},
                                                                                                               'source': {'type': 'keyword'},
                                                                                                               'title': {'type': 'text'},

## 검증 인덱스 생성

In [50]:
eval_vector_db = OpenSearchVectorSearch(
    index_name= eval_index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

In [51]:
%%time

eval_vector_db.add_documents(documents = chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000)


CPU times: user 226 ms, sys: 10.9 ms, total: 237 ms
Wall time: 11.3 s


['dd900c44-530b-4353-997e-c4d8ed1dda66',
 '62e1028a-5af1-4ec1-bcb8-f17199209d3a',
 'b80d8eed-c084-4a3a-a707-0c7b5b0f0143',
 '82bf40f6-c5f7-43ef-b4f5-371c5f00be45',
 '425c4cbc-5059-4134-874d-486939107231',
 '059e4fc4-32fd-48f0-a4de-59d0471fff07',
 'ebaa09f5-751f-4bfc-8a8f-68b14a983893',
 '4721036d-c146-4d38-8342-8e84df55018e',
 'ead37fe0-c714-42ea-b767-02f8cb84c720',
 'ac9d7fef-10f7-415c-b3da-5baa0405f6e4',
 'edc3eb77-d9a0-49e4-885a-b0b86312aeff',
 '1519a69b-89ab-42dc-8448-1c8e5b3027d5',
 'b6a881a2-395e-475f-8e27-f1ffbae667ff',
 '366acede-e763-4a56-bc03-9eda3624121d',
 '7bc79efc-3cca-4785-91bd-8088d27e2ab6',
 'c6162e64-3450-4315-9f32-2587e4ecfd05',
 'c23c03cb-b79f-4d4a-b962-1076e576daf8',
 '47f1acf1-85ee-44aa-9cf6-12d12ecf44d1',
 'a6838dfd-af7e-431a-87a7-886b501995df',
 'bf392ab7-123e-42bc-bf94-d4ecc02ec8cc',
 '4adc1014-57ff-46dc-a9d9-9f567b0d4f1e',
 '873932d1-aa07-4934-9998-e250bb858864',
 'd3053676-c36f-4c00-8360-3222bb0ae64e',
 'f939d1df-2db6-41b0-98f8-7408967a6579',
 'c4deef0c-f76a-

In [53]:
from local_utils.proc_docs import search_hybrid

In [54]:
%%time


filter01 = "[]"
filter02 = "[]"

# query = "how to add image"
query = "이미지 추가 하는 방법"

search_hybrid_result = search_hybrid(
    query=query,
    vector_db= eval_vector_db,
    k=3,
    index_name= eval_index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    Semantic_Search = False,    
    Lexical_Search = False,    
    Hybrid_Search = True,     
    minimum_should_match = 75,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[.5, .5], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=True
)



Query: 
 이미지 추가 하는 방법
##############################
similar_docs_semantic
##############################

Score: 1.0
['. 4.작업을 클릭하고 수행하려는 작업을 선택합니다.또한 장치 추가에 대한 자세한 내용은 장치 추가를 참조하십시오.기기 로그 보기 기기 로그에는 기기에서 발생한 이벤트 목록이 표시됩니다.각 이벤트에 대해 결과 디바이스 상태가 타임스탬프와 함께 표시됩니다. 1.Knox 관리 포털에 로그인하세요.왼쪽 사이드바에서 Knox E-FOTA를 클릭합니다. 2.디바이스 > 모든 디바이스로 이동합니다. 3.보려는 장치의 장치 ID를 클릭합니다. 4.장치 세부 정보 팝업에서 장치 로그 보기를 클릭합니다.장치에 태그 지정 태깅은 장치에 레이블을 지정하고 장치 목록에서 쉽게 검색하려는 경우에 유용합니다.예를 들어, 비즈니스 기능 또는 장치 사용자의 위치를 기준으로 장치에 레이블을 지정할 수 있습니다.이 섹션에서는 단일 장치에 태그를 지정하는 방법을 설명합니다.한 번에 여러 장치에 동일한 태그를 지정하려면 장치에서 일반적인 작업 수행을 참조하십시오. 1.Knox 관리 포털에 로그인하세요.왼쪽 사이드바에서 Knox E-FOTA를 클릭합니다. 2.디바이스 > 모든 디바이스로 이동합니다. 3.태그하려는 장치의 장치 ID를 클릭합니다. 4.장치 세부 정보 대화 상자의 태그 필드에 장치에 연결할 레이블을 입력합니다.예: 판매. 5.저장을 클릭합니다.']
{'source': 'all_processed_data_ko.json', 'seq_num': 25, 'title': '디바이스 관리', 'url': 'https://docs.samsungknox.com/admin/efota-one/features/manage-devices', 'project': 'EFOTA', 'last_updated': '2023-09-06'}
-------------------------------------

# A. Reference

- [Building a RAG AI with OpenSearch Serverless and LangChain](https://caylent.com/blog/building-a-rag-with-open-search-serverless-and-lang-chain)